# Day 9: Rope Bridge

In [1]:
from dataclasses import dataclass
from dataclasses import field
from typing import List
from typing import Optional
import os

In [2]:
@dataclass(frozen=True)
class Point:
  x: int
  y: int

  def __add__(self, other):
    return Point(self.x + other.x, self.y + other.y)

  def __sub__(self, other):
    return Point(self.x - other.x, self.y - other.y)

  def __eq__(self, other):
    return self.x == other.x and self.y == other.y

  def __repr__(self):
    return f'P({self.x}, {self.y})'

  def offset(self, relative: str):
    """ `relative` is a string like 'R 5' which gives a direction
    and a magnitude. The letter can be one of:
      Left: `L` for negative x
      Right: `R` for positive x
      Left: `U` for positive y
      Left: `D` for negative y
    """
    direction, magnitude = relative.split()
    unit = (0,0)
    match direction:
      case 'L':
        unit = (-1,0)
      case 'R':
        unit = (1,0)
      case 'U':
        unit = (0,1)
      case 'D':
        unit = (0,-1)
    magnitude = int(magnitude)
    return Point(self.x + unit[0] * magnitude, self.y + unit[1] * magnitude)

@dataclass
class Vector:
  start: Point
  end: Point
  all_points: Optional[List[Point]] = field(init=False, default=None)

  def __repr__(self):
    return f'V({self.start}, {self.end})'

@dataclass
class Path:
  segments: List[Vector]

  def end_point(self) -> Point:
    if len(self.segments):
      return self.segments[-1].end
    return Point(0,0)

  def __repr__(self):
    return f'{self.segments}'

_Exercise classes_

In [3]:
def exercise_classes():
  path  = Path([])
  path_end = Point(0,0)
  for p in [Point(0, 0), Point(-5, 0), Point(0, 6), Point(10, 20)]:
    for q in ['L 1', 'R 1', 'U 1', 'D 1', 'L 5']:
      print(f'{p} + {q} = {p.offset(q)}, V = {Vector(p, p.offset(q))}')
      next_segment = Vector(path.end_point(), path.end_point().offset(q))
      path.segments.append(next_segment)
  print(f'Path {path}')

exercise_classes()

P(0, 0) + L 1 = P(-1, 0), V = V(P(0, 0), P(-1, 0))
P(0, 0) + R 1 = P(1, 0), V = V(P(0, 0), P(1, 0))
P(0, 0) + U 1 = P(0, 1), V = V(P(0, 0), P(0, 1))
P(0, 0) + D 1 = P(0, -1), V = V(P(0, 0), P(0, -1))
P(0, 0) + L 5 = P(-5, 0), V = V(P(0, 0), P(-5, 0))
P(-5, 0) + L 1 = P(-6, 0), V = V(P(-5, 0), P(-6, 0))
P(-5, 0) + R 1 = P(-4, 0), V = V(P(-5, 0), P(-4, 0))
P(-5, 0) + U 1 = P(-5, 1), V = V(P(-5, 0), P(-5, 1))
P(-5, 0) + D 1 = P(-5, -1), V = V(P(-5, 0), P(-5, -1))
P(-5, 0) + L 5 = P(-10, 0), V = V(P(-5, 0), P(-10, 0))
P(0, 6) + L 1 = P(-1, 6), V = V(P(0, 6), P(-1, 6))
P(0, 6) + R 1 = P(1, 6), V = V(P(0, 6), P(1, 6))
P(0, 6) + U 1 = P(0, 7), V = V(P(0, 6), P(0, 7))
P(0, 6) + D 1 = P(0, 5), V = V(P(0, 6), P(0, 5))
P(0, 6) + L 5 = P(-5, 6), V = V(P(0, 6), P(-5, 6))
P(10, 20) + L 1 = P(9, 20), V = V(P(10, 20), P(9, 20))
P(10, 20) + R 1 = P(11, 20), V = V(P(10, 20), P(11, 20))
P(10, 20) + U 1 = P(10, 21), V = V(P(10, 20), P(10, 21))
P(10, 20) + D 1 = P(10, 19), V = V(P(10, 20), P(10, 19))
P(10,

In [4]:
def load_data(filename : str) -> Path:
  p = Path([])
  with open(filename) as f:
    for line in f.readlines():
      line = line.strip()
      if not len(line) or line[0] == '#':
        continue
      e = p.end_point()
      p.segments.append(Vector(e, e.offset(line)))
  return p

In [5]:
def tween(v: Vector) -> List[Point]:
  """ Expands the given path into a list of all the points it passes through. 
  """
  if v.all_points is not None:
    return v.all_points
  expanded : List[Point]= []
  x_inc = 0
  if v.end.x > v.start.x:
    x_inc = 1
  elif v.end.x < v.start.x:
    x_inc = -1
  y_inc = 0
  if v.end.y > v.start.y:
    y_inc = 1
  elif v.end.y < v.start.y:
    y_inc = -1
  inc = Point(x_inc, y_inc)
  location = v.start
  expanded.append(location)
  while location != v.end:
    location += inc
    expanded.append(location)
  return expanded

In [6]:
def make_tail(head : Path) -> Path:
  """ Creates the path of all points for the tail to follow `head` around. 
  """

  def follow(h : Point, t : Point) -> Point:
    """
      Called after the head has moved. `h` is in the new position and `t` is
      in the previous position. Since the distance was 1 the tail can only be
      in specific positions relative to the head:

      qq1pp
      q000p
      10H01
      r000s
      rr1ss

      Legend:
        * 0, or H requires no move
        * 1 requires the obvious move of 1 along a single axis
        * p, q, r, or s require a diagonal move toward H
    """
    inc = Point(0, 0)

    match h - t:
      case (Point(1, 0) | Point(1, 1) | Point(0, 1) | Point(-1, 1)
       | Point(-1, 0) | Point(-1, -1) | Point(0, -1) | Point(1, -1)
       | Point(0, 0)):
        inc = Point(0, 0)
      case Point(-2, 0):
        inc = Point(-1, 0)
      case Point(2, 0):
        inc = Point(1, 0)
      case Point(0, -2):
        inc = Point(0, -1)
      case Point(0, 2):
        inc = Point(0, 1)
      # p
      case Point(-2, -1) | Point(-2, -2) | Point(-1, -2):
        inc = Point(-1, -1)
      # q
      case Point(2, -1) | Point(2, -2) | Point(1, -2):
        inc = Point(1, -1)
      # r
      case Point(2, 1) | Point(2, 2) | Point(1, 2):
        inc = Point(1, 1)
      # s
      case Point(-2, 1) | Point(-2, 2) | Point(-1, 2):
        inc = Point(-1, 1)
    return t + inc

  tail = Path([])
  for s in head.segments:
    tail_current = tail.end_point()
    tail_segment_points : List[Point] = []
    s.all_points = tween(s)
    for head_current in s.all_points:
      tail_next = follow(head_current, tail_current)
      tail_segment_points.append(tail_next)
      tail_current = tail_next
    tail_segment = Vector(tail_segment_points[0], tail_segment_points[-1])
    tail_segment.all_points=tail_segment_points
    tail.segments.append(tail_segment)
  return tail

In [7]:
def unique_positions(path : Path):
  # Find unique positions visited by the tail
  uniques = set()
  for v in path.segments:
    expanded_points = tween(v)
    for e in expanded_points:
      uniques.add(e)
  return uniques

In [8]:
def get_testdata():
  return [os.path.join('testdata', t) for t in ['easy.txt', 'stay.txt', 'short.txt', 'sample.txt']]

In [9]:
def exercise_unique_positions():
  for f in get_testdata():
    h = load_data(f)
    t = make_tail(h)
    u = unique_positions(t)
    print(f'{f}: Path {t}, {len(u)} unique {u}')  

exercise_unique_positions()

testdata/easy.txt: Path [V(P(0, 0), P(0, 9)), V(P(0, 9), P(0, 19)), V(P(0, 19), P(9, 20)), V(P(9, 20), P(-9, 20))], 39 unique {P(7, 20), P(-4, 20), P(-1, 20), P(-2, 20), P(0, 2), P(0, 5), P(0, 8), P(0, 14), P(0, 11), P(0, 17), P(9, 20), P(0, 20), P(2, 20), P(-9, 20), P(6, 20), P(-7, 20), P(4, 20), P(0, 1), P(0, 7), P(0, 4), P(0, 10), P(0, 16), P(-5, 20), P(0, 13), P(0, 19), P(8, 20), P(-3, 20), P(1, 20), P(0, 0), P(0, 3), P(0, 9), P(3, 20), P(0, 6), P(0, 12), P(5, 20), P(-6, 20), P(0, 15), P(0, 18), P(-8, 20)}
testdata/stay.txt: Path [V(P(0, 0), P(0, 0)), V(P(0, 0), P(0, 0)), V(P(0, 0), P(0, 0)), V(P(0, 0), P(0, 0))], 1 unique {P(0, 0)}
testdata/short.txt: Path [V(P(0, 0), P(13, 0)), V(P(13, 0), P(14, -15)), V(P(14, -15), P(14, -15)), V(P(14, -15), P(23, -16))], 38 unique {P(14, -5), P(4, 0), P(14, -1), P(14, -2), P(8, 0), P(10, 0), P(15, -16), P(1, 0), P(14, -15), P(14, -12), P(12, 0), P(14, -3), P(14, -9), P(17, -16), P(14, -6), P(3, 0), P(19, -16), P(22, -16), P(5, 0), P(21, -16), P

In [10]:
def solver():
  h = load_data('input.txt')
  t = make_tail(h)
  print(f'{len(unique_positions(t))}')  
  
solver()

6522


_Render_

Visualizes the rope as a sequence of frames kind of like the ones shown on the site.

In [11]:
def path_extents(path : Path):
  """ Returns a x,y tuple of Points of the top left and bottom
   right extreme coordinates for the path.
  """
  all_x : List[int] = [s.end.x for s in path.segments]
  all_y : List[int] = [s.end.y for s in path.segments]
  top_left = Point(min(*all_x, 0), max(*all_y, 0))
  bottom_right = Point(max(*all_x, 0), min(*all_y, 0))
  print(f'{top_left}, {bottom_right}')
  return (top_left, bottom_right)

def ascii_render_frame(label : str, point : Point, frame : List[List[str]]):
  """ Fills in the locations of the given vector on the given
      text grid. The text grid must be big enough.
  """
  frame[point.y][point.x] = label[0]

def new_frame(width: int, height: int) -> List[List[str]]:
  return [['.'] * width for r in range(height)]

def ascii_render(head: Path, tail: Path):
  # If either path covers more area then it's `head`.
  extents = path_extents(head)
  frame_width = abs(extents[1].x - extents[0].x) + 1
  frame_height = abs(extents[1].y - extents[0].y) + 1
  frame = new_frame(frame_width, frame_height)
  # Flip the y-offset since the rendering y-axis is inverted.
  offset = Point(-extents[0].x, extents[1].y)
  ascii_render_frame('S', Point(0,0) - offset, frame)
  for step in range(len(head.segments)):
    print(f'{step}: H {head.segments[step]} T {tail.segments[step]}')
    head_points = [*tween(head.segments[step]), head.segments[step].end]
    tail_points = [tail.segments[step].start, *tween(tail.segments[step])]
    while len(tail_points) < len(head_points):
      tail_points.append(tail_points[-1])
    for h, t in zip(head_points, tail_points, strict=True):
      ascii_render_frame('T', t - offset, frame)
      ascii_render_frame('H', h - offset, frame)
      print(h,t)
      for r in frame:
        print(''.join(r))
      print()
      ascii_render_frame('#', t - offset, frame)
      ascii_render_frame('#', h - offset, frame)


In [12]:
def exercise_ascii_render():
  for f in get_testdata():
    h = load_data(f)
    t = make_tail(h)
    ascii_render(h, t)

exercise_ascii_render()

P(-10, 20), P(10, 0)
0: H V(P(0, 0), P(0, 10)) T V(P(0, 0), P(0, 9))
P(0, 0) P(0, 0)
...........H.........
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................

P(0, 1) P(0, 0)
...........T.........
...........H.........
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
..................

In [13]:
def exercise_unique_positions():
  for f in get_testdata():
    h = load_data(f)
    t = make_tail(h)
    print(f'{f}: Path {t}, unique {unique_positions(t)}')  
  
exercise_unique_positions()

testdata/easy.txt: Path [V(P(0, 0), P(0, 9)), V(P(0, 9), P(0, 19)), V(P(0, 19), P(9, 20)), V(P(9, 20), P(-9, 20))], unique {P(7, 20), P(-4, 20), P(-1, 20), P(-2, 20), P(0, 2), P(0, 5), P(0, 8), P(0, 14), P(0, 11), P(0, 17), P(9, 20), P(0, 20), P(2, 20), P(-9, 20), P(6, 20), P(-7, 20), P(4, 20), P(0, 1), P(0, 7), P(0, 4), P(0, 10), P(0, 16), P(-5, 20), P(0, 13), P(0, 19), P(8, 20), P(-3, 20), P(1, 20), P(0, 0), P(0, 3), P(0, 9), P(3, 20), P(0, 6), P(0, 12), P(5, 20), P(-6, 20), P(0, 15), P(0, 18), P(-8, 20)}
testdata/stay.txt: Path [V(P(0, 0), P(0, 0)), V(P(0, 0), P(0, 0)), V(P(0, 0), P(0, 0)), V(P(0, 0), P(0, 0))], unique {P(0, 0)}
testdata/short.txt: Path [V(P(0, 0), P(13, 0)), V(P(13, 0), P(14, -15)), V(P(14, -15), P(14, -15)), V(P(14, -15), P(23, -16))], unique {P(14, -5), P(4, 0), P(14, -1), P(14, -2), P(8, 0), P(10, 0), P(15, -16), P(1, 0), P(14, -15), P(14, -12), P(12, 0), P(14, -3), P(14, -9), P(17, -16), P(14, -6), P(3, 0), P(19, -16), P(22, -16), P(5, 0), P(21, -16), P(7, 0), 